This is intended to be a scratchpad for measures of between-subject SST inhibition.

What are some approaches here?



### SST as a measure of inhibition

Our subject-level behavioral measure is SSRT. But it seems like Post-Error slowing is another important measure. So what about subject-level?



 - Similarity of successful no-go trials to successful go trials
 - Similarity of successful no-go trials to failed no-go trials
 
 
 
 - Similarity of NoGo trials

For each measure we want to:

1. validate it works by showing whether we can do within-run LOOCV to identify nogo and go trials, on average, across the subject group
2. Reduce to a numerical value for each subject for between-subject comparison
3. Compare to ohter measures of inhibition
4. predict

### SST: regressing events directly on inhibition measures measures

So, we can regress No-Go trials on measures of inhibition including both trial level and subject-level values. These will be quite different processes.

#### Subject level

To regress at the subject level you would probably (?) take the average subject image and regress across subjects. This would be fairly straightforward. Then, you'd regress on:

 - various measures of TESQ-E
 - Other measures of inhibition? Bis11?
 - behavioral measures of the SST including mean session SSRT and PES

Some basic steps to do the subject-level stuff:
    
1) Do a single regressor across trials for each trial type
2) Across subjects, predict each of our measures of inhibition for each trial contrast we're interested in
3) Finish

That's really quite basic from an ML point of view but it might be all we're looking for here.

#### Trial level

To regress at the trial level, we need a behavioral measure of inhibition that we can get for each trial, or a subset of trials. I'm not sure how this fits into the wider schema. Let's say we did have, for each subject, a signature that predicted their SSRT or PES. What would we do with it?

We might measure distance between those signatures and the subject's 'Go' trials. We could then predict SST. I think that would be more difficult though, and we should start with the subject-level analysis.

### Basic subject-level

Let's try to imitate what we've already done at the subject level.

In [4]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath("../../ml/"))

from apply_loocv_and_save import *
import gc
import nibabel as nib

nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"
test_train_df = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20210601T183243.csv")

all_sst_events= pd.read_csv(ml_data_folderpath +"/SST/" + "all_sst_events.csv")


dataset_name = 'conditions'
brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_' + dataset_name + '_40subs.pkl'

with open(brain_data_filepath, 'rb') as pkl_file:
    Brain_Data_allsubs = pickle.load(pkl_file)
    
    


from nilearn.decoding import DecoderRegressor, Decoder

script_path = '/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml'
data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/"
# HRF 2s



python initialized for apply_loocv_and_save


/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


4


In [5]:

def condition_resp_trans_func(X):
    return(X.condition_label)


scaffold arguments for function in following paragraph

In [6]:
#set vars for the function

brain_data_filepath = brain_data_filepath
train_test_markers_filepath = data_path + "train_test_markers_20210601T183243.csv"
subjs_to_use = 40
response_transform_func=condition_resp_trans_func
decoder = Decoder(scoring='accuracy',cv=LeaveOneGroupOut(),verbose=1)
mask=None

In [7]:
results_filepath=data_path + "SST/train_test_results_" + dataset_name + "_40subs.pkl"

### Designing a function 

the following block can be turned into a function:

In [21]:
from nilearn.decoding import DecoderRegressor

preprocessed_data = load_and_preprocess(brain_data_filepath,train_test_markers_filepath,subjs_to_use,response_transform_func)

train_y = preprocessed_data['y']
train_X = preprocessed_data['X']
train_groups = preprocessed_data['groups']

if decoder is None:
    decoder = decoderConstructor(
            standardize= True,cv=LeaveOneGroupOut(),
            mask=mask,
            n_jobs=cpus_to_use,verbose=1)

print("starting LeaveOneOut")
#in this design, we're actually dealing with groups
#we select group IDs and then grab the subjects
#so we don't need to use LeaveOneGroupOut
#the grouping is implicit

print("finished preprocessing")


checked for intersection and no intersection between the brain data and the subjects was found.
there were 40 subjects overlapping between the subjects marked for train data and the training dump file itself.
CorrectGo      40
FailedStop     40
Cue            40
CorrectStop    37
FailedGo       26
Name: condition_label, dtype: int64
CorrectGo      40
FailedStop     40
Cue            40
CorrectStop    37
FailedGo       26
Name: condition_label, dtype: int64
test_train_set: 9549
pkl_file: 168
train_test_markers_filepath: 141
brain_data_filepath: 139
response_transform_func: 136
sys: 72
Brain_Data_allsubs: 48
subjs_to_use: 28
False    183
Name: condition_label, dtype: int64


/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/apply_loocv_and_save.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None


183
183
using 40 subjects
starting LeaveOneOut
finished preprocessing


In [22]:
print(decoder)

Decoder(cv=LeaveOneGroupOut(), scoring='accuracy', verbose=1)


In [23]:



print('running one more time on whole dataset for beta map')
decoder.fit(
y=train_y,X=train_X,groups=train_groups)

train_score = decoder.score(train_X,train_y)
train_y_pred = decoder.predict(train_X)



running one more time on whole dataset for beta map
[NiftiMasker.fit] Loading data from Nifti1Image(
shape=(97, 115, 97, 183),
affine=array([[   2.,    0.,    0.,  -96.],
       [   0.,    2.,    0., -132.],
       [   0.,    0.,    2.,  -78.],
       [   0.,    0.,    0.,    1.]])
)
[NiftiMasker.fit] Computing the mask
[NiftiMasker.fit] Resampling mask
[NiftiMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(97, 115, 97, 183),
affine=array([[   2.,    0.,    0.,  -96.],
       [   0.,    2.,    0., -132.],
       [   0.,    0.,    2.,  -78.],
       [   0.,    0.,    0.,    1.]])
)
[NiftiMasker.transform_single_imgs] Extracting region signals
[NiftiMasker.transform_single_imgs] Cleaning extracted signals


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed: 47.1min finished


[NiftiMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(97, 115, 97, 183),
affine=array([[   2.,    0.,    0.,  -96.],
       [   0.,    2.,    0., -132.],
       [   0.,    0.,    2.,  -78.],
       [   0.,    0.,    0.,    1.]])
)
[NiftiMasker.transform_single_imgs] Extracting region signals
[NiftiMasker.transform_single_imgs] Cleaning extracted signals
[NiftiMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(97, 115, 97, 183),
affine=array([[   2.,    0.,    0.,  -96.],
       [   0.,    2.,    0., -132.],
       [   0.,    0.,    2.,  -78.],
       [   0.,    0.,    0.,    1.]])
)
[NiftiMasker.transform_single_imgs] Extracting region signals
[NiftiMasker.transform_single_imgs] Cleaning extracted signals


In [24]:
if type(decoder)==DecoderRegressor:
    weight_img = decoder.coef_img_['beta']
else:
    weight_img = decoder.coef_img_

print('finished learning')

with open(results_filepath, 'wb') as handle:
    pickle.dump({
        'decoder':decoder,
        'weight_img':weight_img,
        'obs':train_y,
        'train_train_score':train_score,
        'train_y_pred': train_y_pred,
        'metadata':preprocessed_data['metadata']
    },handle)


finished learning


In [ ]:
print("finished")

END FUNCTION ################################################################################################

### Function this was modelled off

In [2]:
  
def apply_single_fit_and_save(
    results_filepath,
    brain_data_filepath = '../data/Brain_Data_2sns_60subs.pkl',
    train_test_markers_filepath = "../data/train_test_markers_20210601T183243.csv",
    subjs_to_use = None, #set this to get a subset, otherwise use all of them.
    response_transform_func = None,
    mask=None
    ):
    
    

    preprocessed_data = load_and_preprocess(brain_data_filepath,train_test_markers_filepath,subjs_to_use,response_transform_func)
    
    train_y = preprocessed_data['y']
    train_X = preprocessed_data['X']
    train_groups = preprocessed_data['groups']
    

    
    from nilearn.decoding import DecoderRegressor
    dRegressor = DecoderRegressor(
        standardize= True,cv=LeaveOneGroupOut(),scoring="r2",
        mask=mask,
        n_jobs=cpus_to_use,
        verbose=1)


    print("starting LeaveOneOut")
    #in this design, we're actually dealing with groups
    #we select group IDs and then grab the subjects
    #so we don't need to use LeaveOneGroupOut
    #the grouping is implicit

    print("finished preprocessing")


    
    print('running one more time on whole dataset for beta map')
    dRegressor.fit(
    y=train_y,X=train_X,groups=train_groups)
    
    train_score = dRegressor.score(train_X,train_y)
    train_y_pred = dRegressor.predict(train_X)

    weight_img = dRegressor.coef_img_['beta']

    print('finished learning')

    with open(results_filepath, 'wb') as handle:
        pickle.dump({
            'regressor':dRegressor,
            'weight_img':weight_img,
            'obs':train_y,
            'train_train_score':train_score,
            'train_y_pred': train_y_pred,
            'metadata':preprocessed_data['metadata']
        },handle)


    print('saved.')
    

### Interpreting the results

In [35]:
with open(results_filepath, 'rb') as handle:
    results = pickle.load(open(results_filepath, 'rb'))

In [37]:
results.keys()

dict_keys(['decoder', 'weight_img', 'obs', 'train_train_score', 'train_y_pred', 'metadata'])

In [38]:
results['metadata']['train_y_pred'] = results['train_y_pred']

In [40]:
results['correct'] = results['metadata']['condition_label']==results['metadata']['train_y_pred']

In [41]:
np.sum(results['correct'])/len(results['correct'])

1.0

OK, so in the train/train, we got 100% correct. but that's not LOOCV is it?

In [45]:
[print(k +'\n length: ' + str(len(v))) for k, v in results['decoder'].cv_scores_.items()]

CorrectGo
 length:40
CorrectStop
 length:40
Cue
 length:40
FailedGo
 length:40
FailedStop
 length:40


[None, None, None, None, None]

In [51]:
for k, v in results['decoder'].cv_scores_.items():
    print(k + " cv scores")
    #we used LOGO CV, so each of these is a report of how it scored at correctly identifying items in each group, given the other 39 groups
    #One would think this would be either a zero or 1 because there's only one item in each group...
    print(" length: " + str(len(v)))
    print(v)

CorrectGo cv scores
 length: 40
[0.4, 0.4, 0.6666666666666666, 0.6, 0.5, 0.8, 1.0, 0.4, 0.2, 1.0, 0.25, 1.0, 0.4, 0.2, 0.8, 0.4, 1.0, 0.4, 0.8, 0.5, 0.25, 0.6, 1.0, 0.75, 0.2, 0.4, 0.4, 1.0, 0.2, 0.75, 0.6, 0.8, 0.2, 0.6, 0.4, 0.8, 0.5, 0.25, 0.6, 0.5]
CorrectStop cv scores
 length: 40
[0.2, 0.2, 0.0, 0.2, 0.0, 0.2, 0.25, 0.2, 0.2, 0.2, 0.25, 0.25, 0.2, 0.2, 0.2, 0.2, 0.0, 0.2, 0.2, 0.25, 0.25, 0.2, 0.25, 0.25, 0.2, 0.2, 0.2, 0.25, 0.2, 0.25, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.25, 0.25, 0.2, 0.25]
Cue cv scores
 length: 40
[0.2, 0.2, 0.3333333333333333, 0.2, 0.25, 0.2, 0.25, 0.2, 0.2, 0.4, 0.25, 0.25, 0.2, 0.2, 0.2, 0.2, 0.3333333333333333, 0.2, 0.2, 0.25, 0.25, 0.4, 0.25, 0.25, 0.2, 0.2, 0.2, 0.25, 0.2, 0.25, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.25, 0.25, 0.2, 0.25]
FailedGo cv scores
 length: 40
[0.2, 0.2, 0.0, 0.2, 0.25, 0.2, 0.0, 0.2, 0.2, 0.4, 0.0, 0.0, 0.2, 0.2, 0.2, 0.2, 0.3333333333333333, 0.2, 0.2, 0.0, 0.0, 0.2, 0.0, 0.0, 0.2, 0.2, 0.2, 0.0, 0.2, 0.0, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.

Measuring "accuracy", so I believe these are, out of ALL the samples in the set, the proportion correctly classified as within the set of all labels. An "0.2" could be achieved by classifying all members of the sample as not the label, whereas we know precisely 1 in each set is the label. So these are not great scores.

I wonder whether we could get more interpretable results by scoring based on precision and recall? 

If we use recall, it'll tell us, for each class, how many were correctly classified as that class. That's useful because if we add up aross the set we can tell how many were correctly classified as the actual class they're in. Because no result can be classified as more than one class, we can be confident, if recall is 100%, precision is also 100%.

In [59]:
results['metadata'].iloc[0:15]

,condition_index,condition_label,raw_beta_description,beta,subject,wave,train_y_pred
0,0,CorrectGo,spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1),beta_0001.nii,DEV005,wave1,CorrectGo
1,1,CorrectStop,spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1),beta_0002.nii,DEV005,wave1,CorrectStop
2,2,FailedStop,spm_spm:beta (0003) - Sn(1) FailedStop*bf(1),beta_0003.nii,DEV005,wave1,FailedStop
3,3,Cue,spm_spm:beta (0004) - Sn(1) Cue*bf(1),beta_0004.nii,DEV005,wave1,Cue
4,4,FailedGo,spm_spm:beta (0005) - Sn(1) FailedGo*bf(1),beta_0005.nii,DEV005,wave1,FailedGo
5,0,CorrectGo,spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1),beta_0001.nii,DEV006,wave1,CorrectGo
6,1,CorrectStop,spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1),beta_0002.nii,DEV006,wave1,CorrectStop
7,2,FailedStop,spm_spm:beta (0003) - Sn(1) FailedStop*bf(1),beta_0003.nii,DEV006,wave1,FailedStop
8,3,Cue,spm_spm:beta (0004) - Sn(1) Cue*bf(1),beta_0004.nii,DEV006,wave1,Cue
9,4,FailedGo,spm_spm:beta (0005) - Sn(1) FailedGo*bf(1),beta_0005.nii,DEV006,wave1,FailedGo


In [57]:
results['decoder'].cv_[0:2]

[(array([  5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,
          18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
          31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
          44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
          57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
          70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
          83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
          96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
         109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
         122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
         135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
         148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160,
         161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173,
         174, 175, 176, 177, 178, 179,

### Repeating test measuring precision

#### new function based on code above

In [63]:
from nilearn.decoding import DecoderRegressor  

def apply_single_fit_and_save(
    results_filepath,
    brain_data_filepath = '../data/Brain_Data_2sns_60subs.pkl',
    train_test_markers_filepath = "../data/train_test_markers_20210601T183243.csv",
    subjs_to_use = None, #set this to get a subset, otherwise use all of them.
    response_transform_func = None,
    decoder = None,
    mask=None,
    cpus_to_use=1
    ):
    
    preprocessed_data = load_and_preprocess(brain_data_filepath,train_test_markers_filepath,subjs_to_use,response_transform_func)

    train_y = preprocessed_data['y']
    train_X = preprocessed_data['X']
    train_groups = preprocessed_data['groups']

    if decoder is None:
        decoder = decoderConstructor(
                standardize= True,cv=LeaveOneGroupOut(),
                mask=mask,
                n_jobs=cpus_to_use,verbose=1)

    print("starting LeaveOneOut")
    #in this design, we're actually dealing with groups
    #we select group IDs and then grab the subjects
    #so we don't need to use LeaveOneGroupOut
    #the grouping is implicit

    print('Fitting decoder')
    decoder.fit(
    y=train_y,X=train_X,groups=train_groups)

    print("predicting")
    train_score = decoder.score(train_X,train_y)
    train_y_pred = decoder.predict(train_X)

    if type(decoder)==DecoderRegressor:
        weight_img = decoder.coef_img_['beta']
    else:
        weight_img = decoder.coef_img_

    print('finished learning')

    print("saving")

    result_package = {
            'decoder':decoder,
            'weight_img':weight_img,
            'obs':train_y,
            'train_train_score':train_score,
            'train_y_pred': train_y_pred,
            'metadata':preprocessed_data['metadata']
        }
    with open(results_filepath, 'wb') as handle:
        pickle.dump(result_package,handle)
        
    return(result_package)


In [43]:

from sklearn.metrics import recall_score
from sklearn.metrics import get_scorer

from  https://nilearn.github.io/modules/generated/nilearn.decoding.Decoder.html

here's how to define a callable scorer:

scoring: str, callable or None, optional. Default: ‘roc_auc’

The scoring strategy to use. See the scikit-learn documentation at https://scikit-learn.org/stable/modules/model_evaluation.html#the-scoring-parameter-defining-model-evaluation-rules If callable, takes as arguments the fitted estimator, the test data (X_test) and the test target (y_test) if y is not None. e.g. scorer(estimator, X_test, y_test)


In [92]:
def recall_multiclass_score(estimator,X_test,y_test):
    print("running recall_multiclass_score")
    y_pred = estimator.predict(X_test)
#     print("y_pred:")
#     print(y_pred)
#     print("y_test:")
#     print(y_test)
    rs = recall_score(y_test,y_pred,average='micro')
#     print("recall result:")
#     print(rs)
    return(rs)
    #return(1)


#recall_multiclass_score(decoder,[0,1,1],[0,0,1])

In [93]:
#set vars for the function

brain_data_filepath = brain_data_filepath
train_test_markers_filepath = data_path + "train_test_markers_20210601T183243.csv"
subjs_to_use = 10
response_transform_func=condition_resp_trans_func
decoder_recall = Decoder(
    scoring=recall_multiclass_score,
    standardize= True,
    cv=LeaveOneGroupOut(),
    verbose=1,n_jobs=10)
mask=None

results_filepath=data_path + "SST/train_test_results_recall_" + dataset_name + "_40subs.pkl"

In [94]:
fit_results = apply_single_fit_and_save(
    results_filepath =results_filepath,
    brain_data_filepath = brain_data_filepath,
    train_test_markers_filepath = train_test_markers_filepath,
    subjs_to_use = subjs_to_use,
    response_transform_func = condition_resp_trans_func,
    decoder = decoder_recall
)

checked for intersection and no intersection between the brain data and the subjects was found.
there were 40 subjects overlapping between the subjects marked for train data and the training dump file itself.
CorrectGo      40
FailedStop     40
Cue            40
CorrectStop    37
FailedGo       26
Name: condition_label, dtype: int64
CorrectGo      40
FailedStop     40
Cue            40
CorrectStop    37
FailedGo       26
Name: condition_label, dtype: int64
test_train_set: 9549
pkl_file: 168
train_test_markers_filepath: 141
brain_data_filepath: 139
response_transform_func: 136
sys: 72
Brain_Data_allsubs: 48
subjs_to_use: 28
False    183
Name: condition_label, dtype: int64


/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/apply_loocv_and_save.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None


183
183
using 10 subjects
starting LeaveOneOut
Fitting decoder
[NiftiMasker.fit] Loading data from Nifti1Image(
shape=(97, 115, 97, 46),
affine=array([[   2.,    0.,    0.,  -96.],
       [   0.,    2.,    0., -132.],
       [   0.,    0.,    2.,  -78.],
       [   0.,    0.,    0.,    1.]])
)
[NiftiMasker.fit] Computing the mask
[NiftiMasker.fit] Resampling mask
[NiftiMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(97, 115, 97, 46),
affine=array([[   2.,    0.,    0.,  -96.],
       [   0.,    2.,    0., -132.],
       [   0.,    0.,    2.,  -78.],
       [   0.,    0.,    0.,    1.]])
)
[NiftiMasker.transform_single_imgs] Extracting region signals
[NiftiMasker.transform_single_imgs] Cleaning extracted signals


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:   12.5s
[Parallel(n_jobs=10)]: Done  50 out of  50 | elapsed:   21.7s finished


predicting
running recall_multiclass_score
[NiftiMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(97, 115, 97, 46),
affine=array([[   2.,    0.,    0.,  -96.],
       [   0.,    2.,    0., -132.],
       [   0.,    0.,    2.,  -78.],
       [   0.,    0.,    0.,    1.]])
)
[NiftiMasker.transform_single_imgs] Extracting region signals
[NiftiMasker.transform_single_imgs] Cleaning extracted signals
[NiftiMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(97, 115, 97, 46),
affine=array([[   2.,    0.,    0.,  -96.],
       [   0.,    2.,    0., -132.],
       [   0.,    0.,    2.,  -78.],
       [   0.,    0.,    0.,    1.]])
)
[NiftiMasker.transform_single_imgs] Extracting region signals
[NiftiMasker.transform_single_imgs] Cleaning extracted signals
finished learning
saving


In [95]:
fit_results['decoder'].cv_scores_

{'CorrectGo': [0.4, 0.4, 1.0, 0.4, 0.5, 0.8, 1.0, 0.2, 0.2, 0.8],
 'CorrectStop': [0.2, 0.2, 0.0, 0.2, 0.0, 0.2, 0.25, 0.2, 0.2, 0.2],
 'Cue': [0.2, 0.2, 0.3333333333333333, 0.2, 0.25, 0.2, 0.25, 0.2, 0.2, 0.2],
 'FailedGo': [0.2,
  0.2,
  0.3333333333333333,
  0.2,
  0.25,
  0.2,
  0.25,
  0.2,
  0.2,
  0.6],
 'FailedStop': [0.2,
  0.2,
  0.3333333333333333,
  0.2,
  0.25,
  0.2,
  0.25,
  0.2,
  0.2,
  0.2]}

In [96]:
fit_results['decoder'].classes_

array(['CorrectGo', 'CorrectStop', 'Cue', 'FailedGo', 'FailedStop'],
      dtype='<U11')

In [97]:
fit_results['decoder']

Decoder(cv=LeaveOneGroupOut(), estimator=LinearSVC(max_iter=10000.0),
        memory=Memory(location=None), n_jobs=10,
        scoring=<function recall_multiclass_score at 0x2aaaf7c9f430>,
        verbose=1)

I can't get a CV score that can be unambiguously interpreted. It's hard to intepret what htese are, and the final score does not implement LOGO.

Therefore I think I need to write my own CV function. We could adapt the one I already wrote, or start from scratch. Can be straightforward; wrap the decoder, do LOOCV, etc